### =====================================================================
### IMPORTACIÓN GENERAL DE LA INFORMACIÓN.
### =====================================================================

In [1]:
## IMPORTACIÓN GENERAL DE LIBRERIAS Y VISUALIZACIÓN DE DATOS (matplotlib y seaborn)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as DT
import warnings
import descartes
import geopandas as gpd
import json
import requests
import geocoder

from sklearn.ensemble import RandomForestRegressor
from shapely.geometry import Point, Polygon
from urllib2 import urlopen

%matplotlib inline
warnings.filterwarnings('ignore')
plt.style.use('default') 
sns.set(style="whitegrid") 
plt.rcParams['figure.figsize'] = (15, 10)

In [2]:
#lEEMOS EL CSV
train = pd.read_csv('../../DATA_TP2/train.csv')
test = pd.read_csv('../../DATA_TP2/test.csv')
respuesta = pd.read_csv('../../DATA_TP2/ejemploRespuesta.csv')

### =====================================================================
### PREPARACIÓN DE DATOS, FEATURE ENGINEERING.
### =====================================================================

In [3]:
train['excelente'] = 0
train.loc[train.descripcion.str.contains('vigilancia', na=False), 'vigilancia'] = 1
train['vigilancia'] = 0
train.loc[train.descripcion.str.contains('excelente', na=False), 'excelente'] = 1
train['esquina'] = 0
train.loc[train.descripcion.str.contains('esquina', na=False), 'esquina'] = 1
train['remodelada'] = 0
train.loc[train.descripcion.str.contains('remodelada', na=False), 'remodelada'] = 1
train['lujo'] = 0
train.loc[train.descripcion.str.contains('lujo', na=False), 'lujo'] = 1

In [4]:
train['lat'].fillna(0, inplace=True)
train['lng'].fillna(0, inplace=True)

In [5]:
train['antiguedad_agrupada'] = 0
train.loc[train.antiguedad <= 5, 'antiguedad_agrupada'] = 0
train.loc[((train.antiguedad > 5) & (train.antiguedad <= 15)), 'antiguedad_agrupada'] = 10
train.loc[((train.antiguedad > 15) & (train.antiguedad <= 25)), 'antiguedad_agrupada'] = 20
train.loc[((train.antiguedad > 25) & (train.antiguedad <= 35)), 'antiguedad_agrupada'] = 30
train.loc[((train.antiguedad > 35) & (train.antiguedad <= 45)), 'antiguedad_agrupada'] = 40
train.loc[((train.antiguedad > 45) & (train.antiguedad <= 55)), 'antiguedad_agrupada'] = 50
train.loc[((train.antiguedad > 55) & (train.antiguedad <= 65)), 'antiguedad_agrupada'] = 60
train.loc[((train.antiguedad > 65) & (train.antiguedad <= 75)), 'antiguedad_agrupada'] = 70
train.loc[((train.antiguedad > 75) & (train.antiguedad <= 85)), 'antiguedad_agrupada'] = 80
train.loc[((train.antiguedad > 85) & (train.antiguedad <= 95)), 'antiguedad_agrupada'] = 90
train.loc[train.antiguedad > 95, 'antiguedad_agrupada'] = 100

In [6]:
train['fecha'] = pd.to_datetime(train['fecha'], infer_datetime_format=True)
train['mes'] = train['fecha'].dt.month
train['dia'] = train['fecha'].dt.dayofweek
train['anio'] = train['fecha'].dt.year

In [7]:
train.loc[train.tipodepropiedad.str.contains('Comercial', na=False), 'tipodepropiedad'] = 'Comercio'
train.loc[train.tipodepropiedad.str.contains('Casa en condominio', na=False), 'tipodepropiedad'] = 'Condominio'

### =====================================================================
### TRATAMIENTO SOBRE TEST.
### =====================================================================

In [8]:
test['excelente'] = 0
test.loc[test.descripcion.str.contains('vigilancia', na=False), 'vigilancia'] = 1
test['vigilancia'] = 0
test.loc[test.descripcion.str.contains('excelente', na=False), 'excelente'] = 1
test['esquina'] = 0
test.loc[test.descripcion.str.contains('esquina', na=False), 'esquina'] = 1
test['remodelada'] = 0
test.loc[test.descripcion.str.contains('remodelada', na=False), 'remodelada'] = 1
test['lujo'] = 0
test.loc[test.descripcion.str.contains('lujo', na=False), 'lujo'] = 1

In [9]:
test['lat'].fillna(0, inplace=True)
test['lng'].fillna(0, inplace=True)

In [10]:
test['antiguedad_agrupada'] = 0
test.loc[test.antiguedad <= 5, 'antiguedad_agrupada'] = 0
test.loc[((test.antiguedad > 5) & (test.antiguedad <= 15)), 'antiguedad_agrupada'] = 10
test.loc[((test.antiguedad > 15) & (test.antiguedad <= 25)), 'antiguedad_agrupada'] = 20
test.loc[((test.antiguedad > 25) & (test.antiguedad <= 35)), 'antiguedad_agrupada'] = 30
test.loc[((test.antiguedad > 35) & (test.antiguedad <= 45)), 'antiguedad_agrupada'] = 40
test.loc[((test.antiguedad > 45) & (test.antiguedad <= 55)), 'antiguedad_agrupada'] = 50
test.loc[((test.antiguedad > 55) & (test.antiguedad <= 65)), 'antiguedad_agrupada'] = 60
test.loc[((test.antiguedad > 65) & (test.antiguedad <= 75)), 'antiguedad_agrupada'] = 70
test.loc[((test.antiguedad > 75) & (test.antiguedad <= 85)), 'antiguedad_agrupada'] = 80
test.loc[((test.antiguedad > 85) & (test.antiguedad <= 95)), 'antiguedad_agrupada'] = 90
test.loc[test.antiguedad > 95, 'antiguedad_agrupada'] = 100

In [11]:
test['fecha'] = pd.to_datetime(test['fecha'], infer_datetime_format=True)
test['mes'] = test['fecha'].dt.month
test['dia'] = test['fecha'].dt.dayofweek
test['anio'] = test['fecha'].dt.year

In [12]:
test.loc[test.tipodepropiedad.str.contains('Comercial', na=False), 'tipodepropiedad'] = 'Comercio'
test.loc[test.tipodepropiedad.str.contains('Casa en condominio', na=False), 'tipodepropiedad'] = 'Condominio'

### =====================================================================
### LIMPIEZA Y COLUMNAS RESULTANTES.
### =====================================================================

In [13]:
## ELIMINAMOS LAS COLUMNAS QUE NO SON NUMÉRICAS.
## PERO TENERLAS PRESENTES PARA HACER UN TRABAJO POSTERIOR.

train.drop('titulo', axis=1, inplace=True)
train.drop('descripcion', axis=1, inplace=True)
train.drop('tipodepropiedad', axis=1, inplace=True)
train.drop('direccion', axis=1, inplace=True)
train.drop('ciudad', axis=1, inplace=True)
train.drop('provincia', axis=1, inplace=True)
train.drop('fecha', axis=1, inplace=True)


test.drop('titulo', axis=1, inplace=True)
test.drop('descripcion', axis=1, inplace=True)
test.drop('tipodepropiedad', axis=1, inplace=True)
test.drop('direccion', axis=1, inplace=True)
test.drop('ciudad', axis=1, inplace=True)
test.drop('provincia', axis=1, inplace=True)
test.drop('fecha', axis=1, inplace=True)

In [14]:
## ELIMINAMOS COLUMNAS NUMÉRICAS QUE CARECEN DE SENTIDO.

train.drop('idzona', axis=1, inplace=True)
train.drop('lat', axis=1, inplace=True)
train.drop('lng', axis=1, inplace=True)

test.drop('idzona', axis=1, inplace=True)
test.drop('lat', axis=1, inplace=True)
test.drop('lng', axis=1, inplace=True)

In [15]:
## LIMPIAMOS LOS VALORES NULOS O NAN QUE NO NOS SIRVEN. (CRITERIO)

train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 22 columns):
id                            240000 non-null int64
antiguedad                    240000 non-null float64
habitaciones                  240000 non-null float64
garages                       240000 non-null float64
banos                         240000 non-null float64
metroscubiertos               240000 non-null float64
metrostotales                 240000 non-null float64
gimnasio                      240000 non-null float64
usosmultiples                 240000 non-null float64
piscina                       240000 non-null float64
escuelascercanas              240000 non-null float64
centroscomercialescercanos    240000 non-null float64
precio                        240000 non-null float64
excelente                     240000 non-null int64
vigilancia                    240000 non-null int64
esquina                       240000 non-null int64
remodelada                    240

In [ ]:
test.info()

### =====================================================================
### ALGORITMOS DE MACHINE LEARNING:
### =====================================================================

### =====================================================================
### RANDOM FOREST.
### =====================================================================

In [17]:
# Segmentamos una parte para entrenar y constatar.
train_2016 = train.loc[train.anio == 2016]
train_PREV = train.loc[train.anio < 2016]

In [18]:
# Label a predecir.
labels_2016 = np.array(train_2016['precio'])
labels_PREV = np.array(train_PREV['precio'])

In [19]:
# Resto de los labels.
train_2016 = train_2016.drop('precio', axis = 1)
train_PREV = train_PREV.drop('precio', axis = 1)

In [20]:
# Lista de columnas.
feature_list = list(train_PREV.columns)

In [21]:
# Numpy array
train_2016 = np.array(train_2016)
train_PREV = np.array(train_PREV)

In [22]:
# Observamos lo que nos queda en cada parte.
print('Training Features Shape:', train_PREV.shape)
print('Training Labels Shape:', labels_PREV.shape)
print('Testing Features Shape:', train_2016.shape)
print('Testing Labels Shape:', labels_2016.shape)

('Training Features Shape:', (145962, 21))
('Training Labels Shape:', (145962,))
('Testing Features Shape:', (94038, 21))
('Testing Labels Shape:', (94038,))


In [23]:
# Armamos el regresor con parámetros por defecto.
rf = RandomForestRegressor(n_estimators = 150, random_state = 75)
# Entrenamos.
rf.fit(train_PREV, labels_PREV);

In [24]:
# Hacemos una predicción.
predictions = rf.predict(train_2016)

# Calculamos el error absoluto.
errors = abs(predictions - labels_2016)

# Imprimimos el error.
print('Error:', round(np.mean(errors), 2), 'grados.')

('Error:', 1095348.22, 'grados.')


In [26]:
# Calculamos el porcentaje de error.
mape = 100 * (errors / labels_2016)

# Calculate la precisión.
accuracy = 100 - np.mean(mape)
print('Precision:', round(accuracy, 2), '%.')

('Precision:', 48.69, '%.')


In [34]:
prediccion = rf.predict(test)

In [38]:
## =================================================================================================
## ARMAMOS EN BASE A LA PREDICCIÓN QUE TENEMOS UN CSV PARA SUBIR A KAGGLE CON EL FORMATO INDICADO.
## =================================================================================================
submission = pd.DataFrame({ 'id': test['id'], 'target': prediccion })
submission.to_csv("submission_grupo34_RandomForest.csv", index=False)